In [27]:
import pandas as pd
import numpy as np

### Erstellen der Ineraction Matrix
----

In [43]:
data = np.array([[1,1,2,3],['A','A','B','B']])
data = data.T

In [48]:
df = pd.DataFrame(data, columns = ['user_id','product_name'])
df

,user_id,product_name
0,1,A
1,1,A
2,2,B
3,3,B


In [49]:
# create interaction matrix
df = df.pivot_table(index='user_id', columns='product_name', aggfunc=len, fill_value=0)

In [50]:
df.head()

product_name,A,B
user_id,,
1,2,0
2,0,1
3,0,1
